In [4]:
import pandas as pd
import mediapipe as mp
import cv2
import math
import joblib
from sklearn.preprocessing import StandardScaler

# Load the trained model
model = joblib.load('gesture_recognition_rf.pkl')

# Initialize MediaPipe and OpenCV
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to calculate the Euclidean distance between two landmarks
def calculate_distance(landmark1, landmark2):
    return math.sqrt((landmark2.x - landmark1.x)**2 + (landmark2.y - landmark1.y)**2 + (landmark2.z - landmark1.z)**2)

# Function to calculate the angle between three landmarks
def calculate_angle(landmark1, landmark2, landmark3):
    vec1 = [landmark2.x - landmark1.x, landmark2.y - landmark1.y, landmark2.z - landmark1.z]
    vec2 = [landmark3.x - landmark2.x, landmark3.y - landmark2.y, landmark3.z - landmark2.z]
    
    dot_product = sum(v1 * v2 for v1, v2 in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(v**2 for v in vec1))
    magnitude2 = math.sqrt(sum(v**2 for v in vec2))
    
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    cosine_angle = dot_product / (magnitude1 * magnitude2)
    angle = math.acos(cosine_angle)
    return math.degrees(angle)

# Real-time prediction
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Rendering results and predicting gesture
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                          mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                          mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
                
                # Prepare the feature vector
                features = []
                
                # Calculate distances and angles
                for i in range(21):
                    for j in range(i+1, 21):
                        distance = calculate_distance(hand_landmarks.landmark[i], hand_landmarks.landmark[j])
                        features.append(distance)
                
                for i in range(21):
                    for j in range(i+1, 21):
                        for k in range(j+1, 21):
                            angle = calculate_angle(hand_landmarks.landmark[i], hand_landmarks.landmark[j], hand_landmarks.landmark[k])
                            features.append(angle)
                
                # Convert features to DataFrame
                features_df = pd.DataFrame([features])
                
                # Predict probabilities
                prediction_probs = model.predict_proba(features_df)[0]
                max_prob = max(prediction_probs)
                
                # Determine the prediction based on probability
                if max_prob >= 0.55:
                    prediction = model.classes_[prediction_probs.argmax()]
                else:
                    prediction = 'None'
                
                # Display prediction on the image
                cv2.putText(image, f'Prediction: {prediction}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Display the image
        cv2.imshow('Hand Gesture Recognition', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


C:\Users\SOORAJ\miniconda3\envs\sooraj_os\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\SOORAJ\miniconda3\envs\sooraj_os\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\SOORAJ\miniconda3\envs\sooraj_os\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\SOORAJ\miniconda3\envs\sooraj_os\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\SOORAJ\miniconda3\envs\sooraj_os\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\SOORAJ